In [1]:
import torch
torch.__version__


'2.4.0+cu121'

In [7]:
scalar = torch.tensor([[[1, 2], [3, 4], [5, 6]]])
print(scalar, scalar.ndim, scalar.shape)

tensor([[[1, 2],
         [3, 4],
         [5, 6]]]) 3 torch.Size([1, 3, 2])


In [8]:
scalar.device

device(type='cpu')

In [10]:
!nvidia-smi

Fri Oct 18 16:43:01 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.107.02             Driver Version: 550.107.02     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060        Off |   00000000:01:00.0  On |                  N/A |
| 33%   32C    P8             N/A /  115W |     670MiB /   8188MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
import torch

start = 0
end = 1
step = 0.02

weight = 0.7
bias = 0.3

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

train_split = int(0.8 * len(X)) # 80% of data used for training set, 20% for testing 
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

(40, 40, 10, 10)

In [15]:
model = torch.nn.Linear(1, 1)
model.weight, model.bias

(Parameter containing:
 tensor([[0.9521]], requires_grad=True),
 Parameter containing:
 tensor([-0.7055], requires_grad=True))

In [16]:
model(X) # forward (derdoor jagen)

tensor([[-0.7055],
        [-0.6864],
        [-0.6674],
        [-0.6483],
        [-0.6293],
        [-0.6103],
        [-0.5912],
        [-0.5722],
        [-0.5531],
        [-0.5341],
        [-0.5150],
        [-0.4960],
        [-0.4770],
        [-0.4579],
        [-0.4389],
        [-0.4198],
        [-0.4008],
        [-0.3817],
        [-0.3627],
        [-0.3437],
        [-0.3246],
        [-0.3056],
        [-0.2865],
        [-0.2675],
        [-0.2484],
        [-0.2294],
        [-0.2104],
        [-0.1913],
        [-0.1723],
        [-0.1532],
        [-0.1342],
        [-0.1151],
        [-0.0961],
        [-0.0771],
        [-0.0580],
        [-0.0390],
        [-0.0199],
        [-0.0009],
        [ 0.0182],
        [ 0.0372],
        [ 0.0562],
        [ 0.0753],
        [ 0.0943],
        [ 0.1134],
        [ 0.1324],
        [ 0.1515],
        [ 0.1705],
        [ 0.1895],
        [ 0.2086],
        [ 0.2276]], grad_fn=<AddmmBackward0>)

In [18]:
with torch.inference_mode():
    print(model(X))

tensor([[-0.7055],
        [-0.6864],
        [-0.6674],
        [-0.6483],
        [-0.6293],
        [-0.6103],
        [-0.5912],
        [-0.5722],
        [-0.5531],
        [-0.5341],
        [-0.5150],
        [-0.4960],
        [-0.4770],
        [-0.4579],
        [-0.4389],
        [-0.4198],
        [-0.4008],
        [-0.3817],
        [-0.3627],
        [-0.3437],
        [-0.3246],
        [-0.3056],
        [-0.2865],
        [-0.2675],
        [-0.2484],
        [-0.2294],
        [-0.2104],
        [-0.1913],
        [-0.1723],
        [-0.1532],
        [-0.1342],
        [-0.1151],
        [-0.0961],
        [-0.0771],
        [-0.0580],
        [-0.0390],
        [-0.0199],
        [-0.0009],
        [ 0.0182],
        [ 0.0372],
        [ 0.0562],
        [ 0.0753],
        [ 0.0943],
        [ 0.1134],
        [ 0.1324],
        [ 0.1515],
        [ 0.1705],
        [ 0.1895],
        [ 0.2086],
        [ 0.2276]])


In [23]:
loss_fn = torch.nn.L1Loss() # MAE loss is same as L1Loss
optimizer = torch.optim.SGD(params=model.parameters(), # parameters of target model to optimize
                            lr=0.01) # learning rate (how much the optimizer should change parameters at each step, higher=more (less stable), lower=less (might take a long time))

In [24]:
torch.manual_seed(42)

# Set the number of epochs (how many times the model will pass over the training data)
epochs = 100

# Create empty loss lists to track values
train_loss_values = []
test_loss_values = []
epoch_count = []

for epoch in range(epochs):
    ### Training

    # Put model in training mode (this is the default state of a model)
    model.train()

    # 1. Forward pass on train data using the forward() method inside 
    y_pred = model(X_train)
    # print(y_pred)

    # 2. Calculate the loss (how different are our models predictions to the ground truth)
    loss = loss_fn(y_pred, y_train)

    # 3. Zero grad of the optimizer
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Progress the optimizer
    optimizer.step()

    ### Testing

    # Put the model in evaluation mode
    model.eval()

    with torch.inference_mode():
      # 1. Forward pass on test data
      test_pred = model(X_test)

      # 2. Caculate loss on test data
      test_loss = loss_fn(test_pred, y_test.type(torch.float)) # predictions come in torch.float datatype, so comparisons need to be done with tensors of the same type

      # Print out what's happening
      if epoch % 10 == 0:
            epoch_count.append(epoch)
            train_loss_values.append(loss.detach().numpy())
            test_loss_values.append(test_loss.detach().numpy())
            print(f"Epoch: {epoch} | MAE Train Loss: {loss} | MAE Test Loss: {test_loss} ")

Epoch: 0 | MAE Train Loss: 0.9071348309516907 | MAE Test Loss: 0.7675976157188416 
Epoch: 10 | MAE Train Loss: 0.7919250130653381 | MAE Test Loss: 0.6328878402709961 
Epoch: 20 | MAE Train Loss: 0.6767148971557617 | MAE Test Loss: 0.4981774687767029 
Epoch: 30 | MAE Train Loss: 0.5615048408508301 | MAE Test Loss: 0.36346712708473206 
Epoch: 40 | MAE Train Loss: 0.44629472494125366 | MAE Test Loss: 0.22875675559043884 
Epoch: 50 | MAE Train Loss: 0.33108454942703247 | MAE Test Loss: 0.09404630213975906 
Epoch: 60 | MAE Train Loss: 0.2158743143081665 | MAE Test Loss: 0.04135196655988693 
Epoch: 70 | MAE Train Loss: 0.13913975656032562 | MAE Test Loss: 0.14073064923286438 
Epoch: 80 | MAE Train Loss: 0.11635442078113556 | MAE Test Loss: 0.18724334239959717 
Epoch: 90 | MAE Train Loss: 0.10718341171741486 | MAE Test Loss: 0.20930823683738708 
